In [ ]:
import numpy as np
import cv2 as cv
import os

import sys

sys.path.append("..")

import json
import matplotlib.pyplot as plt

# import apriltag
from scipy.spatial.transform import Rotation

import lac.params as params
from lac.utils.frames import (
    apply_transform,
    invert_transform_mat,
    get_cam_pose_rover,
    OPENCV_TO_CAMERA_PASSIVE,
)
from lac.perception.vision import FiducialLocalizer
from lac.utils.visualization import overlay_tag_detections

%load_ext autoreload
%autoreload 2

# AprilTag detection


In [ ]:
fid_localizer = FiducialLocalizer()

data_path = os.path.expanduser("../../output/lander_fiducials_right_light")
json_data = json.load(open(f"{data_path}/data_log.json"))
lander_pose = np.array(json_data["lander_pose_world"])
poses = np.array([frame["pose"] for frame in json_data["frames"]])

In [ ]:
i = 63
CAM_NAME = "FrontLeft"
img = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)

detections = fid_localizer.detect(img)
overlay = overlay_tag_detections(img, detections)
plt.imshow(overlay)

In [ ]:
i = 1759
CAM_NAME = "Right"
img = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)

detections = fid_localizer.detect(img)
overlay = overlay_tag_detections(img, detections)
plt.imshow(overlay)

# Analyze outliers


# Solving for camera pose using PnP


In [ ]:
from lac.perception.pnp import get_tag_corners_world, solve_tag_pnp

In [ ]:
fid_localizer.estimate_rover_pose(img, CAM_NAME, lander_pose)

In [ ]:
rover_pose = np.array(json_data["frames"][i]["pose"])
rover_pose

In [ ]:
get_tag_corners_world(detections[0].tag_id, lander_pose)

In [ ]:
success, rvec, tvec = cv.solvePnP(
    objectPoints=get_tag_corners_world(detections[0].tag_id, lander_pose),
    imagePoints=detections[0].corners,
    cameraMatrix=params.CAMERA_INTRINSICS,
    distCoeffs=None,
    flags=cv.SOLVEPNP_ITERATIVE,
)
if success:
    print("Rotation Vector (rvec):", rvec)
    print("Translation Vector (tvec):", tvec)
    # Convert rotation vector to rotation matrix
    R, _ = cv.Rodrigues(rvec)
    # Compute homogeneous transformation matrix
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = tvec.flatten()
    print("Homogeneous Transformation Matrix:\n", T)
    # Compute distance
    distance = np.linalg.norm(tvec)  # Euclidean distance
    print(f"Distance to AprilTag: {distance:.2f} meters")

In [ ]:
w_T_c = invert_transform_mat(T)
w_T_c[:3, :3] = OPENCV_TO_CAMERA_PASSIVE @ w_T_c[:3, :3]
print(w_T_c)

In [ ]:
r_T_c = get_cam_pose_rover(CAM_NAME)
c_T_r = invert_transform_mat(r_T_c)

w_T_r = w_T_c @ c_T_r
print(w_T_r)